In [ ]:
#package installs
#E:\Python310\python.exe -m pip install --upgrade pip
#!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile
!E:\Python310\Scripts\pip3.exe install ffmpeg av
#!E:\Python310\python.exe -m pip uninstall torch torchvision torchaudio -y
!E:\Python310\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [3]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [4]:
## Video Processing 1 ##
import pandas as pd
import json
import copy
import csv
import moviepy
from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import pickle

def process_video_to_clips(filesuf_mp4, recalc = False):
    #get number after # but before space afterwards
    str_epnum_temp = filesuf_mp4.split('#')[1]
    str_epnum = str_epnum_temp.split(' ')[0]
    int_epnum = int(str_epnum)
    
    #generate filepaths
    filepath_mp4 = ENV_FOLDER_DATA + filesuf_mp4
    print('Episode='+ str_epnum + ' at ' + filepath_mp4)
    folderpath_normmp4 = ENV_FOLDER_DATA_PROC + 'norm\\' + str_epnum + '\\'
    os.makedirs(folderpath_normmp4, exist_ok=True)
    filepath_normmp4 = folderpath_normmp4 + str_epnum + '.mp4'

    folderpath_eps = ENV_FOLDER_DATA_PROC + 'eps\\' + str_epnum + '\\'
    os.makedirs(folderpath_eps, exist_ok=True)
    folderpath_clips = ENV_FOLDER_DATA_PROC + 'clips\\' + str_epnum + '\\'
    os.makedirs(folderpath_clips, exist_ok=True)
    folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
    os.makedirs(folderpath_clipinfo, exist_ok=True)
    folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
    os.makedirs(folderpath_sourcedf, exist_ok=True)
    #filepath_mp3 = folderpath_eps + str_epnum + '.mp3'
    #filepath_wav = folderpath_eps + str_epnum + '.wav'
    #folderpath_wavsplit = folderpath_eps + 'wavsplit\\'
    #os.makedirs(folderpath_wavsplit, exist_ok=True)
    filepath_segmentation = ENV_FOLDER_DATA_PROC + 'segmentation\\' + str_epnum + '.psv'
    filepath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\' + str_epnum + '.pickle'
    filepath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' + str_epnum + '.pickle'
    filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'

    #all the clips in a video
    filepath_targetclipinfo = folderpath_clipinfo + str_epnum + '.pickle'

    

    #open dict of speakers
    
    json_speakers = {}
    with open(filepath_speakers) as f:
        json_speakers = json.load(f)
    
    #determine speakers to retain
    list_speakers_keep = []
    #skip episodes not included in json_speakers
    if str_epnum not in json_speakers.keys():
        return 0

    for key,val in json_speakers[str_epnum].items():
        if val == 'Joe Rogan':
            continue
        if val == 'Misc':
            continue
        if val == 'Young Jaime':
            continue
        list_speakers_keep.append(key)

    #declare default dictionary
    dict_default = {}
    dict_default['epnum'] = int_epnum
    dict_default['seg'] = 0
    dict_default['sec_start'] = 0.0

    #use cutoffs to split wav file into sections
    #https://stackoverflow.com/questions/51622865/break-up-a-wav-file-by-timestamp

    listdict_data = []
    #read csv file
    with open(filepath_segmentation, newline='\n') as csv_file:
        reader = csv.reader(csv_file, delimiter='|')
        for row in reader:
            dict_temp = {}
            dict_temp['str_segment'] = row[0]
            dict_temp['speaker'] = row[1]
            dict_temp['sec_start'] = row[2]
            dict_temp['sec_end'] = row[3]
            listdict_data.append(dict_temp)
    
    
    #create normalized master video file
    if not os.path.exists(filepath_normmp4):
        clip_total = VideoFileClip(filepath_mp4).without_audio().resize(height=360)
        clip_total = clip_total.fx(vfx.blackwhite)
        clip_total.write_videofile(filepath_normmp4, fps=10)

    #iterate through each segment
    listdict_clips = []
    for val in listdict_data:

        str_segment = val['str_segment']
        int_segment = int(str_segment)
        speaker = val['speaker']
        sec_start = float(val['sec_start']) 
        sec_end = float(val['sec_end'])
        filepath_clip = folderpath_clips + str_segment + '.mp4'
        duration = sec_end - sec_start

        #skip if it's less than 1 second
        if duration < 1.0:
            continue

        #skip data if it's Joe Rogan or Misc
        if speaker not in list_speakers_keep:
            continue

        dict_temp = copy.deepcopy(dict_default)
        dict_temp['seg'] = int_segment
        dict_temp['duration'] = duration
        dict_temp['sec_start'] = sec_start
        dict_temp['sec_end'] = sec_end
        dict_temp['filesuf_clip'] = str_segment + '.mp4'
        dict_temp['filepath_clip'] = filepath_clip

        #create clip
        ffmpeg_extract_subclip(filepath_normmp4, sec_start, sec_end, targetname=filepath_clip)

        #save dictionary
        listdict_clips.append(dict_temp)

    #save data
    
    with open(filepath_targetclipinfo, 'wb') as file:
        pickle.dump(listdict_clips, file, protocol=pickle.HIGHEST_PROTOCOL)
        
    
    #end loop through data
    
    




In [ ]:
filesuf_mp4 = 'Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4'
process_video_to_clips(filesuf_mp4)

In [ ]:
#create clips for episodes post-550
import glob
for path_mp4 in glob.glob(ENV_FOLDER_DATA + '*.mp4'):
    filesuf_mp4 = os.path.basename(path_mp4)
    str_epnum_temp = filesuf_mp4.split('#')[1]
    str_epnum = str_epnum_temp.split(' ')[0]
    int_epnum = int(str_epnum)

    if int_epnum < 550:
        continue

    filesuf_mp4 = os.path.basename(path_mp4)
    process_video_to_clips(filesuf_mp4, recalc = False)

#342 minutes

In [5]:
## Video Processing 1 ##
import pandas as pd
import json
import copy
import csv
import pickle
import torchvision
import itertools
import torch

def process_clips_to_frames(filesuf_mp4, recalc = False):
    #get number after # but before space afterwards
    str_epnum_temp = filesuf_mp4.split('#')[1]
    str_epnum = str_epnum_temp.split(' ')[0]
    int_epnum = int(str_epnum)
    
    #generate filepaths
    filepath_mp4 = ENV_FOLDER_DATA + filesuf_mp4
    print('Episode='+ str_epnum + ' at ' + filepath_mp4)
    folderpath_normmp4 = ENV_FOLDER_DATA_PROC + 'norm\\' + str_epnum + '\\'
    os.makedirs(folderpath_normmp4, exist_ok=True)
    filepath_normmp4 = folderpath_normmp4 + str_epnum + '.mp4'

    #folderpath_eps = ENV_FOLDER_DATA_PROC + 'eps\\' + str_epnum + '\\'
    #os.makedirs(folderpath_eps, exist_ok=True)
    folderpath_clips = ENV_FOLDER_DATA_PROC + 'clips\\' + str_epnum + '\\'
    os.makedirs(folderpath_clips, exist_ok=True)
    folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
    os.makedirs(folderpath_clipinfo, exist_ok=True)
    folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
    os.makedirs(folderpath_sourcedf, exist_ok=True)
    filepath_sourcedf = folderpath_sourcedf + str_epnum + '.pickle'
    #filepath_mp3 = folderpath_eps + str_epnum + '.mp3'
    #filepath_wav = folderpath_eps + str_epnum + '.wav'
    #folderpath_wavsplit = folderpath_eps + 'wavsplit\\'
    #os.makedirs(folderpath_wavsplit, exist_ok=True)
    #filepath_segmentation = ENV_FOLDER_DATA_PROC + 'segmentation\\' + str_epnum + '.psv'
    #filepath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\' + str_epnum + '.pickle'
    filepath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' + str_epnum + '.pickle'
    filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'

    #all the clips in a video
    filepath_targetclipinfo = folderpath_clipinfo + str_epnum + '.pickle'

    
    listdict_clips = []
    #read pickle file
    #if pickle file is empty, skip
    if not os.path.exists(filepath_targetclipinfo):
        return

    with open(filepath_targetclipinfo, 'rb') as file:
        listdict_clips = pickle.load(file)
    

    #iterate through each segment
    #create the object that will be loaded into the neural network
    dict_sourceframes = {}
    for val in listdict_clips:

        int_segment = val['seg']
        sec_start = float(val['sec_start']) 
        sec_end = float(val['sec_end'])
        filepath_clip = val['filepath_clip'] 
        duration = val['duration']

        # https://pytorch.org/vision/main/generated/torchvision.io.VideoReader.html
        # https://pytorch.org/vision/main/auto_examples/others/plot_video_api.html#sphx-glr-auto-examples-others-plot-video-api-py 
        # if you have an error here, download the latest toolkit:
        # https://developer.nvidia.com/cuda-downloads
        torchvision.set_video_backend('pyav')
        reader = torchvision.io.VideoReader(filepath_clip, "video")
        reader.set_current_stream("video")
        transform = torchvision.transforms.Grayscale()

        video_frames = torch.empty(0)
        frames = [] 
        ct = 1
        for frame in reader:
            tempframe = transform(frame['data'])
            frames.append(tempframe)
            ct += 1
        #for x in range(40):
        #    frames.append( torch.empty((1,360,640), dtype=torch.int8) )
        #if len(frames) > 0:
        #    video_frames = torch.stack(frames[:40], 0)
        
        dict_sourceframes[int_segment] = frames

    #save data
    with open(filepath_sourcedf, 'wb') as file:
        pickle.dump(dict_sourceframes, file, protocol=pickle.HIGHEST_PROTOCOL)
    
    #end loop through data

In [11]:
filesuf_mp4 = 'Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4'
process_clips_to_frames(filesuf_mp4)

Episode=568 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4


In [7]:
#create clips for episodes post-550
import glob
for path_mp4 in glob.glob(ENV_FOLDER_DATA + '*.mp4'):
    str_epnum_temp = path_mp4.split('#')[1]
    str_epnum = str_epnum_temp.split(' ')[0]
    int_epnum = int(str_epnum)
    #print(int_epnum)
    if int_epnum < 550:
        continue

    #print(filesuf_mp4)
    filesuf_mp4 = os.path.basename(path_mp4)
    process_clips_to_frames(filesuf_mp4, recalc = False)
#runs for 65 minutes

Episode=550 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #550 - Rupert Sheldrake.mp4
Episode=551 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #551 - Graham Hancock.mp4
Episode=552 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #552 - Kid Cudi.mp4
Episode=553 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #553 - Thaddeus Russell.mp4
Episode=554 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #554 - W. Kamau Bell.mp4
Episode=555 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #555 - Rory Albanese.mp4
Episode=556 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #556 - Bryan Callen.mp4
Episode=557 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #557 - Bryan Callen.mp4
Episode=558 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experience #558 - Honey Honey.mp4
Episode=559 at E:\W4732 Computer Vision\Final Paper Data\Joe Rogan Experien

In [12]:
#5 seconds to open
#8GB
dict_sourceframes = {}
with open('E:\\W4732 Computer Vision\\Final Paper Data Proc\\sourcedf\\568.pickle', 'rb') as file:
    dict_sourceframes = pickle.load(file)
print(dict_sourceframes.keys())

dict_keys([8, 9, 10, 11, 12, 16, 18, 21, 22, 24, 28, 29, 31, 33, 34, 35, 36, 37, 39, 41, 44, 45, 46, 47, 48, 49, 52, 55, 56, 57, 60, 61, 62, 64, 65, 66, 70, 71, 72, 75, 77, 78, 79, 94, 98, 103, 104, 107, 115, 116, 118, 119, 121, 122, 124, 125, 126, 131, 132, 133, 134, 135, 137, 138, 140, 141, 142, 145, 154, 155, 156, 157, 158, 159, 161, 164, 166, 171, 173, 174, 175, 176, 177, 178, 180, 181, 182, 184, 185, 188, 189, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 240, 243, 245, 248, 249, 252, 253, 256, 257, 260, 261, 262, 263, 264, 265, 266, 267, 268, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296, 297, 299, 300, 301, 302, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 316, 317, 321, 322, 324, 325, 326, 327, 328, 329, 330, 331, 333, 3

In [13]:
dict_sourceframes[8]

tensor([[[[ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          ...,
          [22, 21, 21,  ...,  0,  0,  0],
          [16, 16, 15,  ...,  0,  0,  0],
          [11, 11, 12,  ...,  0,  0,  0]]],


        [[[ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          ...,
          [22, 21, 21,  ...,  0,  0,  0],
          [16, 16, 15,  ...,  0,  0,  0],
          [11, 11, 12,  ...,  0,  0,  0]]],


        [[[ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          ...,
          [18, 18, 18,  ...,  0,  0,  0],
          [14, 14, 14,  ...,  0,  0,  0],
          [12, 12, 12,  ...,  0,  0,  0]]],


        ...,


        [[[ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          ...,
          [18, 18, 18,  ...,  0